In [27]:
using DrWatson
@quickactivate "sst-mot-analysis"

import DrWatson: datadir, srcdir
import DrWatson: @quickactivate

using GH19, TMI, PythonCall, Revise
using CSV, DataFrames, NaNStatistics, NCDatasets

In [37]:
function get_MOT_and_SST(TMIfile::String, γ::Grid)
    θ = readfield(TMIfile, "θ", γ); #true mean
    volume = cellvolume(γ).tracer 
    area = cellarea(γ).tracer 

    θ̄_MOT = nansum(θ.tracer .* volume) / nansum(volume)
    θ_SST = getsurfaceboundary(θ);
    θ̄_SST = nansum(θ_SST.tracer .* area) / nansum(area)
    return θ̄_MOT, θ̄_SST
end

function get_MOT_and_SST(θ::Field, γ::Grid)
    volume = cellvolume(γ).tracer 
    area = cellarea(γ).tracer 

    θ̄_MOT = nansum(θ.tracer .* volume) / nansum(volume)
    θ_SST = getsurfaceboundary(θ);
    θ̄_SST = nansum(θ_SST.tracer .* area) / nansum(area)
    return θ̄_MOT, θ̄_SST
end

get_MOT_and_SST (generic function with 2 methods)

In [41]:


###### LOAD LGM ###########
TMI_version = "LGM_90x45x33_G14";
A, Alu, γ_lgm, TMIfile, L, B = config(TMI_version); println(sum(cellvolume(γ_lgm))) 
TMIfile = TMI.pkgdatadir("TMI_"*TMI_version*".nc")

θ̄_LGM_MOT, θ̄_LGM_SST = get_MOT_and_SST(TMIfile, γ_lgm);

###### LOAD MODERN ###########
TMI_version = "modern_90x45x33_G14_v2";
A, Alu, γ_mod, TMIfile, L, B = config(TMI_version); 
TMIfile = TMI.pkgdatadir("TMI_"*TMI_version*".nc")
θ̄_mod_MOT, θ̄_mod_SST_fine = get_MOT_and_SST(TMIfile, γ_mod);

###### LOAD MODERN GH19 ###########
TMI_version = GH19.TMIversion()
A, Alu, γ_mod, TMIfile, L, B = config(TMI_version); 

OPT15_ds = NCDataset(datadir("Theta_OPT-0015.nc"))
idx_90s = findmin(abs.(OPT15_ds["year"][:] .- 1990))[2]
GH19_theta = GH19.readfield_snapshot(datadir("Theta_OPT-0015.nc"), "theta", idx_90s, γ_mod)
θ̄_mod_MOT_90s, θ̄_mod_SST_fine_90s = get_MOT_and_SST(GH19_theta, γ_mod);



Form water-mass matrix A
  0.020721 seconds (1.99 k allocations: 26.437 MiB)
LU factorization of A
  0.580567 seconds (88 allocations: 305.217 MiB, 16.20% gc time)
circulation matrix L=
  0.004192 seconds (333 allocations: 26.828 KiB)
Boundary matrix B=
  0.002232 seconds (332 allocations: 26.805 KiB)
1.4033505303988716e18
Form water-mass matrix A
  0.026768 seconds (4.57 k allocations: 34.897 MiB)
LU factorization of A
  0.821807 seconds (89 allocations: 492.957 MiB, 0.54% gc time)
circulation matrix L=
  0.004475 seconds (802 allocations: 66.539 KiB)
Boundary matrix B=
  0.003870 seconds (801 allocations: 66.516 KiB)
Form water-mass matrix A
  0.044618 seconds (5.23 k allocations: 118.943 MiB)
LU factorization of A
  5.321390 seconds (89 allocations: 2.810 GiB, 0.35% gc time)
circulation matrix L=
  0.130313 seconds (8.14 M allocations: 224.520 MiB, 1.69% gc time)
Boundary matrix B=
  0.013752 seconds (6.14 k allocations: 7.270 MiB)


In [42]:
θ̄_mod_MOT_90s - θ̄_LGM_MOT

2.499487474729789

In [43]:
θ̄_mod_MOT - θ̄_LGM_MOT

2.526746048139418

In [ ]:
θ̄_mod_SST_fineθ̄_mod_MOT - θ̄_LGM_MOTθ̄_mod_MOT - θ̄_LGM_MOT - θ̄_LGM_SST

5.358732182544513

2.526746048139418

2.526746048139418